In [66]:
import numpy as np
from copy import deepcopy
import random

In [67]:
# Tile wall index values
TILE_WALL_VERTICAL = 0
TILE_WALL_HORIZONTAL = 1
TILE_WALL_CORNER_BOTTOM_LEFT = 2
TILE_WALL_CORNER_BOTTOM_RIGHT = 3
TILE_WALL_CORNER_TOP_RIGHT = 4
TILE_WALL_CORNER_TOP_LEFT = 5
TILE_FLOOR = 6
TILE_DOOR_HORIZONTAL = 7
TILE_DOOR_VERTICAL = 8

In [68]:
#Constants for genetic algorithm parameters
POPULATION_SIZE = 100
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
NUM_ROOMS = 3 #Target number of rooms

In [69]:
class LevelGenerationProblem:
    """Handles map size, door positions, and fitness evaluation."""

    def __init__(self, width, height, num_rooms):
        self.width = width  # Map width (N)
        self.height = height  # Map height (N)
        self.num_rooms = num_rooms  # Number of rooms
        self.door_positions = self.generate_door_positions()  # Fixed immutable door positions

    def generate_door_positions(self):
        """
        Generate immutable door positions for the map.
        Door positions are restricted to N-2 x N-2 (to avoid edges).
        """
        positions = []
        for _ in range(self.num_rooms - 1):  # Generate `num_rooms - 1` doors
            x = random.randint(1, self.width - 2)
            y = random.randint(1, self.height - 2)
            orientation = random.choice([TILE_DOOR_HORIZONTAL, TILE_DOOR_VERTICAL])  # Random door type
            positions.append((x, y, orientation))
        return positions

    def decode_chromosome(self, chromosome):
        """Insert doors into the chromosome to build the full map."""
        map_with_doors = deepcopy(chromosome)

        # Add predefined doors while maintaining immutability
        for x, y, orientation in self.door_positions:
            map_with_doors[y, x] = orientation

        return map_with_doors

    def calculate_fitness(self, chromosome):
        """
        Fitness evaluation for a map focusing on valid walls and connected enclosures.
        """
        map_with_doors = self.decode_chromosome(chromosome)
        fitness = 0

        # Track which tiles have been evaluated
        visited = set()

        # Iterate over all tiles to validate rooms
        for y in range(self.height):
            for x in range(self.width):
                if (x, y) not in visited and map_with_doors[y, x] in [
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_VERTICAL,
                    TILE_DOOR_HORIZONTAL,
                    TILE_DOOR_VERTICAL,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                    TILE_WALL_CORNER_TOP_RIGHT,
                    TILE_WALL_CORNER_TOP_LEFT,
                ]:
                    # Validate the enclosure starting from this tile
                    is_valid, score, tiles = self.validate_enclosure(map_with_doors, x, y)
                    visited.update(tiles)
                    fitness += score if is_valid else -score  # Add or penalize based on validity

        return fitness

    def validate_enclosure(self, tile_map, start_x, start_y):
        """
        Validate an enclosure starting from a given tile using BFS/DFS.
        - Ensure all walls/doors are validly connected.
        """
        queue = [(start_x, start_y)]
        tiles = set()  # Track tiles belonging to this enclosure
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Left, Right, Up, Down

        is_valid = True  # Assume it's valid until proven otherwise
        score = 0  # Cumulative score for this enclosure

        while queue:
            x, y = queue.pop(0)
            if (x, y) in tiles or not (0 <= x < self.width and 0 <= y < self.height):
                continue

            tiles.add((x, y))
            tile = tile_map[y, x]

            # Evaluate the current tile based on its type
            if tile in [TILE_WALL_HORIZONTAL, TILE_DOOR_HORIZONTAL]:
                valid, contributions = self.validate_horizontal_tile(tile_map, x, y)
            elif tile in [TILE_WALL_VERTICAL, TILE_DOOR_VERTICAL]:
                valid, contributions = self.validate_vertical_tile(tile_map, x, y)
            elif tile == TILE_WALL_CORNER_BOTTOM_LEFT:
                valid, contributions = self.validate_bottom_left_corner(tile_map, x, y)
            elif tile == TILE_WALL_CORNER_BOTTOM_RIGHT:
                valid, contributions = self.validate_bottom_right_corner(tile_map, x, y)
            elif tile == TILE_WALL_CORNER_TOP_RIGHT:
                valid, contributions = self.validate_top_right_corner(tile_map, x, y)
            elif tile == TILE_WALL_CORNER_TOP_LEFT:
                valid, contributions = self.validate_top_left_corner(tile_map, x, y)
            else:
                valid, contributions = False, -1  # Invalid tile

            is_valid = is_valid and valid  # Reduce validity if any part is invalid
            score += contributions

            # Add neighbors to the queue
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.width and 0 <= ny < self.height and (nx, ny) not in tiles:
                    queue.append((nx, ny))

        return is_valid, score, tiles

    def validate_horizontal_tile(self, tile_map, x, y):
        """
        Validate a horizontal wall or door tile by checking its left and right neighbors.
        Horizontal doors contribute additional positive score compared to walls.
        """
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_HORIZONTAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_HORIZONTAL

        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_TOP_LEFT]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_WALL_CORNER_TOP_RIGHT]

        is_valid = valid_left and valid_right
        # Add extra score if it's a horizontal door
        base_score = 10
        score = base_score + 10 if tile_map[y, x] == TILE_DOOR_HORIZONTAL else base_score
        score = score if is_valid else -3

        return is_valid, score

    def validate_vertical_tile(self, tile_map, x, y):
        """
        Validate a vertical wall or door tile by checking its top and bottom neighbors.
        Vertical doors contribute additional positive score compared to walls.
        """
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_VERTICAL
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_VERTICAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_TOP_RIGHT]
        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_BOTTOM_RIGHT]

        is_valid = valid_above and valid_below
        # Add extra score if it's a vertical door
        base_score = 10
        score = base_score + 10 if tile_map[y, x] == TILE_DOOR_VERTICAL else base_score
        score = score if is_valid else -3

        return is_valid, score


    def validate_bottom_left_corner(self, tile_map, x, y):
        """Validate a bottom-left corner based on its neighbors."""
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_VERTICAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_HORIZONTAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT]

        is_valid = valid_above and valid_right
        score = 10 if is_valid else -3
        return is_valid, score

    def validate_bottom_right_corner(self, tile_map, x, y):
        """Validate a bottom-right corner based on its neighbors."""
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_VERTICAL
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_HORIZONTAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_RIGHT]
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT]

        is_valid = valid_above and valid_left
        score = 10 if is_valid else -3
        return is_valid, score

    def validate_top_right_corner(self, tile_map, x, y):
        """Validate a top-right corner based on its neighbors."""
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_VERTICAL
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_HORIZONTAL

        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_RIGHT]
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT]

        is_valid = valid_below and valid_left
        score = 10 if is_valid else -3
        return is_valid, score

    def validate_top_left_corner(self, tile_map, x, y):
        """Validate a top-left corner based on its neighbors."""
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_VERTICAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_HORIZONTAL

        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT]

        is_valid = valid_below and valid_right
        score = 10 if is_valid else -3
        return is_valid, score


In [70]:
class LevelIndividual:
    """Represents an individual in the genetic algorithm."""

    def __init__(self, problem):
        self.problem = problem
        self.chromosome = self.randomize_chromosome()  # Chromosome is an N x N 2D tile matrix

    def randomize_chromosome(self):
    # Create random chromosome selecting from all tile types
        chromosome = np.zeros((self.problem.height, self.problem.width), dtype=int)

        for y in range(self.problem.height):
            for x in range(self.problem.width):
                # Randomly assign any possible wall type, corners, doors
                chromosome[y, x] = random.choice([
                    TILE_WALL_VERTICAL,
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_CORNER_TOP_LEFT,
                    TILE_WALL_CORNER_TOP_RIGHT,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                ])

        return chromosome


    def mutate(self, mutation_rate):
        """
        Mutate the chromosome by randomly changing tiles, excluding doors.
        """
        for _ in range(int(self.problem.width * self.problem.height * mutation_rate)):
            x = random.randint(0, self.problem.width - 1)
            y = random.randint(0, self.problem.height - 1)

            # Ensure doors remain immutable
            if (x, y) not in [(door[0], door[1]) for door in self.problem.door_positions]:
                self.chromosome[y, x] = random.choice([
                    TILE_WALL_VERTICAL,
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                    TILE_WALL_CORNER_TOP_LEFT,
                    TILE_WALL_CORNER_TOP_RIGHT
                ])

    def crossover(self, other_parent):
        """
        Perform crossover with another parent.
        Mix parts of the two maps.
        """
        child1 = deepcopy(self)
        child2 = deepcopy(other_parent)

        # Single-point crossover on the 2D matrix
        split_row = random.randint(0, self.problem.height - 1)

        child1.chromosome[:split_row, :] = self.chromosome[:split_row, :]
        child1.chromosome[split_row:, :] = other_parent.chromosome[split_row:, :]

        child2.chromosome[:split_row, :] = other_parent.chromosome[:split_row, :]
        child2.chromosome[split_row:, :] = self.chromosome[split_row:, :]

        return child1, child2


In [71]:
class Parameters:
    def __init__(self, population_size=POPULATION_SIZE, max_generations=MAX_GENERATIONS, mutation_rate=MUTATION_RATE):
        self.population_size = population_size
        self.max_generations = max_generations
        self.mutation_rate = mutation_rate

In [72]:
problem = LevelGenerationProblem(width=20, height=15, num_rooms=3)
# View the generated door positions
print("Door positions:", problem.door_positions)


Door positions: [(7, 13, 8), (6, 1, 8)]


In [76]:
ind1 = LevelIndividual(problem)

In [77]:
# Decode the individual's chromosome into a map
decoded_map = problem.decode_chromosome(ind1.chromosome)
print("Decoded Map:")
print(decoded_map)


Decoded Map:
[[4 1 4 2 5 5 4 1 3 5 1 2 5 2 3 2 3 4 5 3]
 [0 3 1 0 1 3 8 5 2 2 0 2 4 2 1 3 0 4 3 1]
 [1 2 1 4 3 0 1 2 4 3 0 3 3 1 0 4 1 3 5 4]
 [2 1 4 1 5 0 4 3 0 1 2 2 1 4 2 0 1 2 1 3]
 [3 2 4 3 2 2 5 1 1 3 5 1 1 5 1 2 5 0 3 5]
 [2 4 2 0 0 4 3 1 0 4 5 0 5 3 1 1 1 5 2 4]
 [4 5 3 2 0 1 2 2 5 0 5 2 1 1 5 4 1 4 5 3]
 [1 4 4 0 1 3 1 5 1 3 1 3 3 3 1 2 0 0 2 2]
 [2 4 2 4 3 3 3 4 5 1 4 3 5 4 5 4 0 5 1 2]
 [2 3 0 4 1 1 1 4 2 3 2 3 4 1 0 3 2 0 5 3]
 [1 4 0 3 3 1 2 2 4 1 3 1 1 4 2 1 5 5 3 1]
 [2 1 0 4 4 0 2 5 0 5 2 0 2 3 2 5 3 3 0 5]
 [1 5 0 0 5 0 4 4 2 1 5 5 5 1 5 2 0 5 5 3]
 [1 1 4 4 1 5 3 8 0 3 4 4 2 1 1 0 3 2 3 5]
 [2 3 4 4 0 3 2 4 0 3 0 3 0 0 4 3 3 1 0 2]]


In [78]:
# Calculate fitness
fitness = problem.calculate_fitness(ind1.chromosome)
print("Fitness:", fitness)


Fitness: 133


In [79]:
def choose_parent(population):
    # Tournament selection
    tournament_size = 5
    tournament = random.sample(population, tournament_size)
    return max(tournament, key=lambda x: x.fitness)

In [ ]:
def run_genetic(problem, params):
    population = [LevelIndividual(problem) for _ in range(params.population_size)]

    for generation in range(params.max_generations):
        population.sort(key=lambda x: x.fitness, reverse=True)
        best_individual = population[0]
        if generation %10 == 0:
            print(f"Generation {generation}: Best fitness = {best_individual.fitness}")

        new_population = [best_individual] #Elitism

        while len(new_population) < params.population_size:
            parent1 = choose_parent(population)
            parent2 = choose_parent(population)
            child1, child2 = parent1.crossover(parent2)
            child1.mutate(params.mutation_rate)
            child2.mutate(params.mutation_rate)
            new_population.extend([child1, child2])
        population = new_population

    return population[0]